## Import libraries

In [ ]:
import src.config as config

## ETL — Extract, Transform, Load of Raw Dataset

This section is responsible for the Extraction phase of the ETL process, pulling historical financial data from multiple sources as defined in the configuration file [dataset.yaml](../configs/dataset.yaml).

- Data Storage: Each dataset is stored locally in the data/raw/ directory, organized and saved individually as .csv files for traceability and versioning.
- Data Cleaning:
  - Missing values are treated using a standard cleaning strategy.
  - Features with low variance (threshold < 0.01) are removed to reduce noise and improve modeling efficiency. In this execution, no low-variance columns were found.

In [ ]:
from src import dataset

dataset.main(
    # ---- REPLACE DEFAULT AS APPROPRIATE ----
    asset = '^BVSP',
    asset_focus = 'Close',
    years = 9
    # -----------------------------------------
    )

## Feature Engineering — Time Series Preparation

This part of the pipeline is responsible for transforming the cleaned dataset into structured features suitable for training time series models. It includes the following key steps:
- Feature Generation: Constructs relevant features based on historical market data.
- Dataset Splitting: The dataset is split into training and testing sets using a consistent strategy to preserve temporal structure.
- Time Series Windowing: Converts the sequential data into overlapping windows, enabling the model to learn temporal dependencies.
- Saving Artifacts: Both training and testing sets are stored for reproducibility, along with the transformation pipelines applied during preprocessing.

In [1]:
from src import config
from src import features

features.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    train_dir = config.PROCESSED_DATA_DIR,
    test_dir = config.PROCESSED_DATA_DIR,
    sequence_length = 20
    # -----------------------------------------
)

2025-06-23 09:30:16.661 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Repositories\ds-lstm-ibov


2025-06-23 09:30:23.256 | INFO     | src.features:main:32 - Generating features from dataset...
2025-06-23 09:30:23.330 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-23 09:30:23.356 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...
2025-06-23 09:30:23.472 | SUCCESS  | src.features:main:47 - Saving train features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-23 09:30:23.625 | SUCCESS  | src.features:main:51 - Saving test features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-23 09:30:23.629 | SUCCESS  | src.features:main:55 - Features generation complete.
2025-06-23 09:30:23.629 | INFO     | src.features:main:57 - Saving transformers...
2025-06-23 09:30:23.630 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-23 09:30:23.635 | INFO     | src.features:main:73 - Total t

## Modeling

The modeling process begins with loading the training dataset. Next, the base model is constructed, and both the compilation and training strategies are defined. With the pipeline structure in place, the model is trained over 100 epochs using an iterative approach to adjust the weights.

During training, key metrics are monitored, including accuracy, loss (error), validation accuracy and loss (on unseen data), as well as the learning rate.

In [2]:
from src import config
from src.modeling import train

train.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    X_path = config.PROCESSED_DATA_DIR / "X_train.npy",
    y_path = config.PROCESSED_DATA_DIR / "y_train.npy",
    batch_size = 16,
    validation_split = 0.1,
    # -----------------------------------------
)

2025-06-23 09:30:26.821 | INFO     | src.modeling.train:main:32 - Loading training dataset...
2025-06-23 09:30:26.855 | INFO     | src.modeling.train:main:42 - Building model...
2025-06-23 09:30:26.855 | INFO     | src.modeling.train:main:48 - Selecting compile strategy...
2025-06-23 09:30:26.855 | INFO     | src.modeling.train:main:51 - Selecting training strategy...
2025-06-23 09:30:26.855 | INFO     | src.modeling.train:main:54 - Building model training pipeline template...
2025-06-23 09:30:26.855 | INFO     | src.modeling.train:main:61 - Training model...
Epoch 1/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - loss: 0.2690 - mae: 0.5757 - mse: 0.7160 - r2_score: -0.1908 - rmse: 0.7489 - smape: 154.8423 - val_loss: 0.3994 - val_mae: 0.7380 - val_mse: 1.1720 - val_r2_score: -0.0058 - val_rmse: 1.0372 - val_smape: 149.9421 - learning_rate: 0.0010
Epoch 2/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.2354 - mae: 0.5253 - mse: 0.6286 - r2_score: 0.0271 - rmse: 0.6912 - smape: 

Traceback (most recent call last):
  File "c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\mlflow\store\tracking\file_store.py", line 329, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\mlflow\store\tracking\file_store.py", line 427, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1373, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\mlflow\store\tracking\file_store.py", line 1366, in _read_helper
    result = read_yaml(root, file_name)
             ^^^^^^^

2025-06-23 09:36:15.948 | INFO     | src.modeling.train:main:70 - Saving 'Sequential_epoch102_loss0.1278.keras' in 'C:\Repositories\ds-lstm-ibov\models'...


2025/06/23 09:36:16 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/06/23 09:36:33 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\SAMSUNG\AppData\Local\Temp\tmpk5du9mgn\model, flavor: keras). Fall back to return ['keras==3.9.2']. Set logging level to DEBUG to see the full traceback. 
2025/06/23 09:36:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'RegressionSimpleModelBuilder' already exists. Creating a new version of this model...
Created version '2' of model 'RegressionSimpleModelBuilder'.


2025-06-23 09:36:38.289 | SUCCESS  | src.modeling.train:main:88 - Modeling training complete.
2025-06-23 09:36:38.289 | INFO     | src.modeling.train:main:91 - Elapsed time: 371.47 seconds


## Predict

In the prediction stage, the trained model is loaded along with the input data. The model then performs inference, generating predictions based on the provided data.


In [3]:
from src.modeling import predict
predict.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    preprocessor_path = config.PROCESSED_DATA_DIR / "preprocessor.pkl",
    model_path = config.MODELS_DIR / "Sequential_epoch36_loss0.1808.keras",  # Select the best model  
    postprocessor_path = config.PROCESSED_DATA_DIR / "postprocessor.pkl",
    output_path = config.PROCESSED_DATA_DIR / "dataset_report.csv",
    # -----------------------------------------
)

2025-06-23 09:36:38.345 | INFO     | src.modeling.predict:main:28 - Performing inference for model...
2025-06-23 09:36:38.404 | INFO     | src.modeling.predict:main:32 - Input data shape: (101, 50)
2025-06-23 09:36:38.467 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...


c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


ValueError: Length mismatch: Expected axis has 81 elements, new values have 1 elements

## Plot

Open the [Power BI Report]((../reports/pbi/amp-fynance.pbip)) and refresh data.